In [1]:
from pathlib import Path
import sys  
import os
import pandas as pd 
from datetime import datetime
import scipy.sparse as sps
from numpy import linalg as LA

import numpy as np
from operator import itemgetter
import optuna as op

In [2]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "libs")))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "src")))

In [3]:
from libs.Recommenders.SLIM.SLIM_BPR_Python import SLIM_BPR_Python
from libs.Evaluation.Evaluator import EvaluatorHoldout

In [4]:
URM_train = sps.load_npz("URM_train_ens.npz")
URM_val = sps.load_npz("URM_val_ens.npz")

In [5]:
evaluator_validation = EvaluatorHoldout(URM_val, cutoff_list=[10])

EvaluatorHoldout: Ignoring 1008 ( 2.8%) Users that have less than 1 test interactions


In [7]:
slim_model_bpr = SLIM_BPR_Python

In [10]:
def objective(trial):
    topK = trial.suggest_int("topK", 250, 350)
    lambda_i = trial.suggest_float("lambda_i", 0.0005, 0.0125)
    lambda_j = trial.suggest_float("lambda_j", 0.00005, 0.00125)
    learning_rate = trial.suggest_float("learning_rate", 0.05, 0.7)
    
    recommender = slim_model_bpr(URM_train)
    recommender.fit(**trial.params)
    
    result_data, _ = evaluator_validation.evaluateRecommender(recommender)
    
    return result_data["MAP"][10]

In [11]:
class SaveResults(object):
    
    def __init__(self):
        self.results_df = pd.DataFrame()
    
    def __call__(self, optuna_study, optuna_trial):
        hyperparam_dict = optuna_trial.params.copy()
        hyperparam_dict["result"] = optuna_trial.values[0]
        
        #self.results_df = pd.concat([self.results_df, pd.DataFrame(hyperparam_dict).transpose()], ignore_index=True)
        self.results_df = self.results_df._append(hyperparam_dict, ignore_index=True)

In [ ]:
study = op.create_study(direction="maximize")
save_results_bpr = SaveResults()
study.optimize(objective, callbacks=[save_results_bpr], n_trials=100)

[I 2024-12-18 00:02:07,637] A new study created in memory with name: no-name-de1582e4-33f9-413d-ac8c-fa0ed659f8be


Epoch 1, Iteration 35736 in 2.38 seconds. Samples per second 14990.63
Epoch 2, Iteration 35736 in 2.42 seconds. Samples per second 14760.53
Epoch 3, Iteration 35736 in 2.62 seconds. Samples per second 13652.81
Epoch 4, Iteration 35736 in 2.31 seconds. Samples per second 15478.18
Epoch 5, Iteration 35736 in 2.33 seconds. Samples per second 15361.44
Epoch 6, Iteration 35736 in 2.46 seconds. Samples per second 14514.58
Epoch 7, Iteration 35736 in 2.72 seconds. Samples per second 13125.72
Epoch 8, Iteration 35736 in 2.72 seconds. Samples per second 13141.48
Epoch 9, Iteration 35736 in 2.54 seconds. Samples per second 14053.89
Epoch 10, Iteration 35736 in 2.61 seconds. Samples per second 13718.04
Epoch 11, Iteration 35736 in 2.38 seconds. Samples per second 14995.86
Epoch 12, Iteration 35736 in 2.33 seconds. Samples per second 15367.32
Epoch 13, Iteration 35736 in 2.34 seconds. Samples per second 15299.22
Epoch 14, Iteration 35736 in 2.30 seconds. Samples per second 15518.94
Epoch 15, Itera

[I 2024-12-18 00:03:48,882] Trial 0 finished with value: 0.01777028988090674 and parameters: {'topK': 318, 'lambda_i': 0.0011157823850022623, 'lambda_j': 0.00017452996482221594, 'learning_rate': 0.19463064094902288}. Best is trial 0 with value: 0.01777028988090674.


Epoch 1, Iteration 35736 in 1.65 seconds. Samples per second 21606.34
Epoch 2, Iteration 35736 in 2.17 seconds. Samples per second 16451.35
Epoch 3, Iteration 35736 in 2.16 seconds. Samples per second 16528.46
Epoch 4, Iteration 35736 in 2.23 seconds. Samples per second 15990.54
Epoch 5, Iteration 35736 in 2.32 seconds. Samples per second 15394.98
Epoch 6, Iteration 35736 in 2.58 seconds. Samples per second 13846.33
Epoch 7, Iteration 35736 in 2.65 seconds. Samples per second 13476.96
Epoch 8, Iteration 35736 in 2.82 seconds. Samples per second 12682.25
Epoch 9, Iteration 35736 in 2.84 seconds. Samples per second 12590.33
Epoch 10, Iteration 35736 in 2.48 seconds. Samples per second 14385.54
Epoch 11, Iteration 35736 in 2.45 seconds. Samples per second 14560.77
Epoch 12, Iteration 35736 in 2.48 seconds. Samples per second 14436.46
Epoch 13, Iteration 35736 in 2.68 seconds. Samples per second 13328.68
Epoch 14, Iteration 35736 in 2.94 seconds. Samples per second 12167.55
Epoch 15, Itera

[I 2024-12-18 00:05:28,347] Trial 1 finished with value: 0.017586674720183798 and parameters: {'topK': 261, 'lambda_i': 0.009593761858407294, 'lambda_j': 0.0010206720856885511, 'learning_rate': 0.42276790880883514}. Best is trial 0 with value: 0.01777028988090674.


...

Epoch 1, Iteration 35736 in 1.38 seconds. Samples per second 25913.30
Epoch 2, Iteration 35736 in 1.50 seconds. Samples per second 23767.21
Epoch 3, Iteration 35736 in 1.74 seconds. Samples per second 20584.05
Epoch 4, Iteration 35736 in 1.71 seconds. Samples per second 20927.25
Epoch 5, Iteration 35736 in 1.76 seconds. Samples per second 20288.88
Epoch 6, Iteration 35736 in 1.78 seconds. Samples per second 20023.77
Epoch 7, Iteration 35736 in 1.87 seconds. Samples per second 19071.39
Epoch 8, Iteration 35736 in 1.81 seconds. Samples per second 19763.56
Epoch 9, Iteration 35736 in 1.86 seconds. Samples per second 19189.99
Epoch 10, Iteration 35736 in 1.86 seconds. Samples per second 19205.39
Epoch 11, Iteration 35736 in 1.91 seconds. Samples per second 18715.39
Epoch 12, Iteration 35736 in 1.87 seconds. Samples per second 19149.77
Epoch 13, Iteration 35736 in 1.91 seconds. Samples per second 18686.27
Epoch 14, Iteration 35736 in 1.86 seconds. Samples per second 19255.20
Epoch 15, Itera

[I 2024-12-18 02:21:23,389] Trial 95 finished with value: 0.017959237868533068 and parameters: {'topK': 252, 'lambda_i': 0.000781364861234363, 'lambda_j': 0.00041131888064671916, 'learning_rate': 0.2103629018272334}. Best is trial 34 with value: 0.018005800406241015.


Epoch 1, Iteration 35736 in 1.41 seconds. Samples per second 25365.13
Epoch 2, Iteration 35736 in 1.52 seconds. Samples per second 23559.66
Epoch 3, Iteration 35736 in 1.71 seconds. Samples per second 20879.08
Epoch 4, Iteration 35736 in 1.74 seconds. Samples per second 20524.43
Epoch 5, Iteration 35736 in 1.77 seconds. Samples per second 20206.16
Epoch 6, Iteration 35736 in 1.82 seconds. Samples per second 19599.42
Epoch 7, Iteration 35736 in 1.82 seconds. Samples per second 19672.40
Epoch 8, Iteration 35736 in 1.82 seconds. Samples per second 19677.23
Epoch 9, Iteration 35736 in 1.85 seconds. Samples per second 19364.29
Epoch 10, Iteration 35736 in 1.87 seconds. Samples per second 19113.46
Epoch 11, Iteration 35736 in 1.86 seconds. Samples per second 19178.26
Epoch 12, Iteration 35736 in 1.87 seconds. Samples per second 19104.10
Epoch 13, Iteration 35736 in 1.87 seconds. Samples per second 19158.07
Epoch 14, Iteration 35736 in 1.88 seconds. Samples per second 19033.68
Epoch 15, Itera

[I 2024-12-18 02:22:43,937] Trial 96 finished with value: 0.017781459450861764 and parameters: {'topK': 252, 'lambda_i': 0.0007814804476491066, 'lambda_j': 0.0004109938877481999, 'learning_rate': 0.21793809520815016}. Best is trial 34 with value: 0.018005800406241015.


Epoch 1, Iteration 35736 in 1.42 seconds. Samples per second 25188.03
Epoch 2, Iteration 35736 in 1.59 seconds. Samples per second 22459.94
Epoch 3, Iteration 35736 in 1.79 seconds. Samples per second 19994.71
Epoch 4, Iteration 35736 in 1.82 seconds. Samples per second 19676.56


In [ ]:
save_results_bpr.results_df.to_csv("result_experiments/SLIM_bpr/results_optuna.csv")